# Implementing Neural Networks

he World Health Organization (WHO)’s Global Health Observatory (GHO) data repository tracks life expectancy for countries worldwide by following health status and many other related factors.

Although there have been a lot of studies undertaken in the past on factors affecting life expectancy considering demographic variables, income composition, and mortality rates, it was found that the effects of immunization and human development index were not taken into account.

This dataset covers a variety of indicators for all countries from 2000 to 2015 including:

immunization factors
mortality factors
economic factors
social factors
other health-related factors
Ideally, this data will eventually inform countries concerning which factors to change in order to improve the life expectancy of their populations. If we can predict life expectancy well given all the factors, this is a good sign that there are some important patterns in the data. Life expectancy is expressed in years, and hence it is a number. This means that in order to build a predictive model one needs to use regression.

In this project, you will design, train, and evaluate a neural network model performing the task of regression to predict the life expectancy of countries using this dataset

Load the life_expectancy.csv dataset into a pandas DataFrame by first importing pandas, and then using the pandas.read_csv() function to load the file and assign the resulting DataFrame to a variable called dataset

In [1]:
import pandas as pd

dataset=pd.read_csv('life_expectancy.csv')

Observe the data by printing the first five entries in the DataFrame dataset by using the DataFrame.head() function. Make sure to see what the columns are and what the data types are. Locate the feature we would like to predict: life expectancy. Use DataFrame.describe() to see the summary statistics of the data.

In [2]:
print(dataset.head())
print(dataset.describe())

       Country  Year      Status  Adult Mortality  infant deaths  Alcohol  \
0  Afghanistan  2015  Developing            263.0             62     0.01   
1  Afghanistan  2014  Developing            271.0             64     0.01   
2  Afghanistan  2013  Developing            268.0             66     0.01   
3  Afghanistan  2012  Developing            272.0             69     0.01   
4  Afghanistan  2011  Developing            275.0             71     0.01   

   percentage expenditure  Hepatitis B  Measles    BMI   ...  \
0               71.279624         65.0      1154   19.1  ...   
1               73.523582         62.0       492   18.6  ...   
2               73.219243         64.0       430   18.1  ...   
3               78.184215         67.0      2787   17.6  ...   
4                7.097109         68.0      3013   17.2  ...   

   Total expenditure  Diphtheria    HIV/AIDS         GDP  Population  \
0               8.16         65.0        0.1  584.259210  33736494.0   
1       

Drop the Country column from the DataFrame using the DataFrame drop method. Why? To create a predictive model, knowing from which country data comes can be confusing and it is not a column we can generalize over. We want to learn a general pattern for all the countries, and not only those dependent on specific countries.

In [3]:
dataset = dataset.drop(['Country'], axis = 1)

labels=dataset.iloc[:,-1]

In the next two steps, you will split the data into labels and features. Labels are contained in the “Life expectancy” column. It’s the final column in the DataFrame. Create a new variable called labels. Use iloc indexing to assign the final column of dataset to it.


Features span from the first column to the last column (not including it, since it’s a label column in our dataset). Create a new variable called features. Use iloc indexing to assign a subset of columns from first to last (not including the last column) to it.

In [4]:
features=dataset.iloc[:,0:-1]

When you observed your dataset you probably noticed that some columns are categorical. We learned in this lesson that categorical columns need to be converted into numerical columns using methods such as one-hot-encoding. Use pandas.get_dummies(DataFrame) to apply one-hot-encoding on all the categorical columns. Assign the result of the encoding back to the features variable.

In [5]:
features=pd.get_dummies(features)

Split your data into training set and test sets using the sklearn.model_selection.train_test_split() function. Assign

the training features to a variable called features_train
training labels to a variable called labels_train
test data to a variable called features_test
test labels to a variable called labels_test.
You can choose any percentage for test_size and any value for random_state.

In [6]:
from sklearn.model_selection import train_test_split

features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.20, random_state=1)


The next step is to standardize/normalize your numerical features. You can pick whichever method you want. In this step, create a sklearn.compose.ColumnTransformer instance called ct to set up the normalization/standardization procedure. When initializing ColumnTransformer make sure to list all of the numerical features you have in your dataset. Or use DataFrame.select_dtypes() to select float64 or int64 feature types automatically.

Fit your instance ct of ColumnTransformer to the training data and at the same time transform it by using the ColumnTransformer.fit_transform() method. Assign the result to a variable called features_train_scaled.

Transform your test data instance features_test using the trained ColumnTransformer instance ct. Assign the result to a variable called features_test_scaled.

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.compose import ColumnTransformer

numerical_features = features.select_dtypes(include=['float64', 'int64'])
numerical_columns = numerical_features.columns
 
ct = ColumnTransformer([("only numeric", StandardScaler(), numerical_columns)], remainder='passthrough')

features_train_scaled=ct.fit_transform(features_train)
features_test_scaled = ct.transform(features_test)

Create an instance of the tensorflow.keras.models.Sequential model called my_model.

Create the input layer to the network model using tf.keras.layers.InputLayer with the shape corresponding to the number of features in your dataset. Assign the resulting input layer to a variable called input.

Add the input layer from the previous step to the model instance my_model.

Add one keras.layers.Dense hidden layer with any number of hidden units you wish. Use the relu activation function.

Add a keras.layers.Dense output layer with one neuron since you need a single output for a regression prediction.


Print the summary of the model using the Sequential.summary() method.

In [8]:
from tensorflow.keras.models import Sequential
my_model = Sequential()
from tensorflow.keras.layers import InputLayer
input = InputLayer(input_shape = (features.shape[1], ))

my_model.add(input)

from tensorflow.keras.layers import Dense
my_model.add(Dense(64, activation = "relu"))
my_model.add(Dense(1))
print(my_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                1408      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 1,473
Trainable params: 1,473
Non-trainable params: 0
_________________________________________________________________
None


Create an instance of the Adam optimizer with the learning rate equal to 0.01.


Once your optimizer is initialized, compile the model using the Sequential.compile() method.

Assign the following parameters:

For loss use the Mean Squared Error (mse)
For metrics use the Mean Absolute Error (mae)
For opt (the optimizer parameters) use the instance of the optimizer you created in the previous step.


In [9]:
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate = 0.1)
my_model.compile(loss = 'mse', metrics = ['mae'], optimizer = opt)

Train your model with the Sequential.fit() method by passing it the following parameters:

your preprocessed training data
training labels
number of epochs equal to 40
batch size equal to 1
verbose set to 1

Using the Sequential.evaluate() method, evaluate your trained model on the preprocessed test data set, and with the test labels. Set verbose to 0. Store the result of the evaluation in two variables: res_mse and res_mae.

Print your final loss (RMSE) and final metric (MAE) to check the predictive performance on the test set.

In [10]:
my_model.fit(features_train_scaled, labels_train, epochs = 40, batch_size = 1, verbose = 1)

res_mse , res_mae = my_model.evaluate(features_test_scaled, labels_test, verbose = 0)

print(res_mse, res_mae)


Epoch 1/40
2350/2350 [==============================] - 5s 2ms/step - loss: 433.7993 - mae: 11.9613
Epoch 2/40
2350/2350 [==============================] - 4s 2ms/step - loss: 740.6522 - mae: 12.1074
Epoch 3/40
2350/2350 [==============================] - 4s 2ms/step - loss: 76.0890 - mae: 5.1209
Epoch 4/40
2350/2350 [==============================] - 3s 1ms/step - loss: 29.3629 - mae: 4.0385
Epoch 5/40
2350/2350 [==============================] - 4s 2ms/step - loss: 141.4147 - mae: 4.7105
Epoch 6/40
2350/2350 [==============================] - 4s 2ms/step - loss: 58.4191 - mae: 4.1903
Epoch 7/40
2350/2350 [==============================] - 3s 1ms/step - loss: 23.4805 - mae: 3.6528
Epoch 8/40
2350/2350 [==============================] - 3s 1ms/step - loss: 23.4287 - mae: 3.6651
Epoch 9/40
2350/2350 [==============================] - 3s 1ms/step - loss: 22.8816 - mae: 3.6766
Epoch 10/40
2350/2350 [==============================] - 3s 1ms/step - loss: 25.9256 - mae: 3.8326
Epoch 11/40
23